In [10]:
#生成/转换/仿制
import torch
import numpy as np

x = torch.Tensor(2, 2); print(f"{x.flatten()} {x.shape}")

data = [[1, 2],[3, 4]]
np_data = np.array(data)
t_data  = torch.tensor(data)

tensor = torch.rand(3, 4)
print(f"tensor.shape:{tensor.shape}, tensor.dtype:{tensor.dtype}, tensor.device:{tensor.device}")

np2t = torch.from_numpy(np_data)
x_ones = torch.ones_like(np2t)                    # retains the properties of x_data
x_rand = torch.rand_like(np2t, dtype=torch.float) # overrides the datatype of x_data

tensor([-1.7182e+22,  4.5852e-41, -1.7182e+22,  4.5852e-41]) torch.Size([2, 2])
tensor.shape:torch.Size([3, 4]), tensor.dtype:torch.float32, tensor.device:cpu


In [17]:
#行列操作
tensor = torch.ones(2, 10)
tensor[:,1] = 0
print(tensor)

tensor([[1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 0., 1., 1., 1., 1., 1., 1., 1., 1.]])


In [5]:
import torch; torch.arange(13).chunk(6)

(tensor([0, 1, 2]),
 tensor([3, 4, 5]),
 tensor([6, 7, 8]),
 tensor([ 9, 10, 11]),
 tensor([12]))

In [22]:
#运算
x = torch.ones((2, 2)) * 2
y = torch.ones((2, 2)) * 3
print(f"x.mul(y)      :{x.mul(y).flatten()}")      # This computes the element-wise product
print(f"x * y         :{(x * y).flatten()}")       # Alternative syntax
print(f"x.matmul(y.T) :{x.matmul(y.T).flatten()}") # Alternative syntax:
print(f"x.mm(y)       :{(x.mm(y.T).flatten())}")
print(f"x @ y.T       :{(x @ y.T).flatten()}")


x.mul(y)      :tensor([6., 6., 6., 6.])
x * y         :tensor([6., 6., 6., 6.])
x.matmul(y.T) :tensor([12., 12., 12., 12.])
x.mm(y)       :tensor([12., 12., 12., 12.])
x @ y.T       :tensor([12., 12., 12., 12.])


In [20]:
x, y = torch.ones(2, 2) * 2, torch.ones(2, 2) * 3
result = torch.empty(2, 2)
torch.add(x, y,out=result)
print(result.flatten())
print(f"x+y={(x.add(y).flatten())} x={x.flatten()}")  #不覆盖加法 
print(f"x+y={(x.add_(y).flatten())} x={x.flatten()}") #覆盖加法
print(f"y*5={(y.mul_(5)).flatten()}")                             #覆盖乘法

"""
x.add_(y), x.mul_(y), x.div_(y); 
x.pow_(y); x.tanh_();
x.t_(); 
x.copy_(y); 
x.clamp_(min=0, max=1)
""" if False else None

tensor([5., 5., 5., 5.])
x+y=tensor([5., 5., 5., 5.]) x=tensor([2., 2., 2., 2.])
x+y=tensor([5., 5., 5., 5.]) x=tensor([5., 5., 5., 5.])
y*5=tensor([15., 15., 15., 15.])


In [19]:
# [Attention]: Tensors on the CPU and NumPy arrays can share their underlying memory locations, 
# and changing one will change the other.
t = torch.ones(10)
print(f"t: {t}")
n = t.numpy()
n[0] = 0
print(f"t: {t}")

t: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
t: tensor([0., 1., 1., 1., 1., 1., 1., 1., 1., 1.])


In [26]:
#random init tensor
from timm.models.layers import trunc_normal_
x = torch.zeros((1, 3, 255, 255))
trunc_normal_(x, mean=.02, std=.02)
print(torch.mean(x), torch.std(x))

tensor(0.0200) tensor(0.0200)


In [ ]:
import torch
x = torch.zeros(4,4,4,4,4)
i = torch.ones(100, dtype=torch.long)
j = torch.ones(100, dtype=torch.long)
print(x[i][j].shape)

In [ ]:
x = torch.arange(0, 4).reshape(2, 2)
i = torch.zeros(2000, dtype=torch.long)
print(x)
print(x[0])
print(x[i])
print(x[i].shape)
#[169, 3] --> [2401, 3] --> [49, 49, 3]

In [7]:
import torch
w_size = 7; num_heads = 12
# torch.zeros((2 * w_size - 1) * (2 * w_size - 1, num_heads))

RuntimeError: Storage size calculation overflowed with sizes=[13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12, 13, 12]

In [3]:
import torch
tensor = torch.randn(10, 10)
tensor = torch.nn.Parameter(tensor)

print(tensor.shape)

torch.Size([10, 10])


In [13]:
import torch
import torchvision

class Model(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        conv1 = torch.nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3, bias=False)
        conv2 = torch.nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3, bias=False)
        conv3 = torch.nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.mix = torch.nn.ModuleList([conv1, conv2, conv3])

model = Model()
fc_params_id = list(map(id, model.modules()))
#print(model.modules())

for i in model.mix.named_children():
    print(i)

def _set_module(model, submodule_key, module):
    tokens = submodule_key.split('.')
    sub_tokens = tokens[:-1]
    cur_mod = model
    for s in sub_tokens:
        cur_mod = getattr(cur_mod, s)
    setattr(cur_mod, tokens[-1], module)

conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

for n, m in model.named_modules():
    if '2' in n:
        _set_module(model, n, conv1)

print(model.get_submodule('mix.2'))

('0', Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False))
('1', Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False))
('2', Conv2d(64, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False))
Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [12]:
import torch
import torch.nn as nn


class MLP(nn.Module):

    def __init__(self, num_in, num_hidden, num_out, seed=None):
        super().__init__()
        dropout_rate = 0.2
        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)

        self.linear1 = nn.Linear(num_in, num_hidden)
        self.linear2 = nn.Linear(num_hidden, num_hidden)
        self.linear3 = nn.Linear(num_hidden, num_out)

        self.sequential_module = nn.Sequential(
            nn.Linear(num_hidden, num_hidden),
            nn.ReLU(),
            nn.Dropout(p=0.2),
        )

        self.relu = nn.ReLU()

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.relu(self.linear1(x))
        x = self.dropout1(x)

        x = self.relu(self.linear2(x))
        x = self.dropout2(x)

        x = self.sequential_module(x)

        x = self.softmax(self.linear3(x))

        return x



# print(torch.__version__)

model = MLP(num_in=2, num_hidden=16, num_out=3)
# print(model)
# print("\n")

relu_lst = [k.split('.') for k, m in model.named_modules(remove_duplicate=False) if 'linear' in k]
print(relu_lst)
for *parent, k in relu_lst:
    print(len(parent), k)
    if len(parent) == 0:
        model.relu = nn.Softplus()
    else:
        model.get_submodule('.'.join(parent))[int(k)] = nn.Softplus()


[['linear1'], ['linear2'], ['linear3']]
0 linear1
0 linear2
0 linear3
